# API Project

In this project, we will work with **A**pplication **P**rogramming **I**nterface (or API, for short). 

I have selected the API named YouTube Data API v3, which is an API that provides access to YouTube data, such as videos, playlists, and channels.

## Table of contents
* Importing required libraries 
* API Key
* Definition of URL and its parameters
* Getting Response from URL
* Extracting useful data from the response
* Storing data in Pandas DataFrame
* Resulting DataFrame
* Exporting the data to CSV file

## Importing required libraries

In [84]:
import json
import pandas as pd
import requests
import os

## API Key

In [85]:
# We will input API Key as temporary Jupyter environment variable
os.environ["YOUTUBE_API_KEY"] = open("YOUTUBE_API_KEY.txt").readlines()[0]

In [86]:
#checking it was successful
%env

{'ACCESS_TOKEN_SECRET_TWITTER': '<hidden>',
 'ACCESS_TOKEN_TWITTER': '<hidden>',
 'ALLUSERSPROFILE': 'C:\\ProgramData',
 'API_KEY_SECRET_TWITTER': '<hidden>',
 'API_KEY_TWITTER': '<hidden>',
 'APPDATA': 'C:\\Users\\yutinghu\\AppData\\Roaming',
 'BEARER_TOKEN_TWITTER': '<hidden>',
 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
 'COMPUTERNAME': 'BCN1-5CG1196TFJ',
 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
 'CONDA_DEFAULT_ENV': 'base',
 'CONDA_EXE': 'C:\\Users\\yutinghu\\Anaconda3\\Scripts\\conda.exe',
 'CONDA_PROMPT_MODIFIER': '(base) ',
 'CONDA_PYTHON_EXE': 'C:\\Users\\yutinghu\\Anaconda3\\python.exe',
 'CONDA_SHLVL': '1',
 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
 'HOMEDRIVE': 'C:',
 'HOMEPATH': '\\Users\\yutinghu',
 'JAVA_TOOLS_OPTIONS': '-Dlog4j2.formatMsgNoLookups=true',
 'JAVA_TOOL_OPTIONS': '-Dlog4j2.formatMsgNoLookups=true

In [87]:
API_KEY = os.getenv("YOUTUBE_API_KEY")

## Definition of URL and its parameters

In [88]:
parameters = {'key': API_KEY,
              'channelId': 'UCWD3Q1-d9vzxt1cXUmpbjqg',
              'part': 'snippet,id',
              'order': 'date',
              'maxResults': 50,
             }
url = "https://www.googleapis.com/youtube/v3/search"

## Getting Response from URL

Now, we will launch the request of information to the API throught GET, and show its response in JSON format

In [89]:
response = requests.get(
    url, 
    params=parameters
)

# checking server status code is successful:
response.status_code

200

In [90]:
response = response.json()

## Extracting useful data from the response

In [91]:
# Checking content of first element within 'items'
response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'mSS-o0m_QYFV4EjiRXypPlVUI_0',
 'id': {'kind': 'youtube#video', 'videoId': '0lRyVSDrY4s'},
 'snippet': {'publishedAt': '2022-06-08T14:59:34Z',
  'channelId': 'UCWD3Q1-d9vzxt1cXUmpbjqg',
  'title': 'De manucure à développeuse web',
  'description': '"C\'est comme apprendre une nouvelle langue". - Elle, c\'est Mathilde et elle vient de quitter son job de manucure dans un salon ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/0lRyVSDrY4s/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/0lRyVSDrY4s/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/0lRyVSDrY4s/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Ironhack',
  'liveBroadcastContent': 'none',
  'publishTime': '2022-06-08T14:59:34Z'}}

Our basis to get useful data will be based on following information:
- **videoID** : video identification code
- **video_title** : video title text
- publishedAt : timestamp at which video was published, from which we will extract both date (**published_date**) and time (**published_time**)
- **video_link**: URL to access each video

In [92]:
# parsing video_id for first element within 'items'
video_id = response['items'][0]['id']['videoId']
video_id

'0lRyVSDrY4s'

In [93]:
# parsing video_title for first element within 'items'
video_title = response['items'][0]['snippet']['title']
video_title

'De manucure à développeuse web'

In [94]:
# parsing published_date for first element within 'items'
published_date = response['items'][0]['snippet']['publishedAt'].split('T')[0]
published_date

'2022-06-08'

In [95]:
# parsing published_time for first element within 'items'
published_time = response['items'][0]['snippet']['publishedAt'].split('T')[1].split('Z')[0]
published_time

'14:59:34'

In [96]:
# getting video_link for first element within 'items'
video_link = 'https://www.youtube.com/watch?v='+video_id
video_link

'https://www.youtube.com/watch?v=0lRyVSDrY4s'

## Storing data in a DataFrame

In [97]:
# Creating function to extract the data we parsed for first element but for all videos in 
# JSON response and storing them in a DataFrame: 

def get_videos(video, ironhack_videos):
    ironhack_videos = pd.DataFrame(columns=['video_id','video_title', 'published_date', 'published_time', 'video_link'])
    for video in response['items']:
        if video['id']['kind'] == "youtube#video":
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            published_date = video['snippet']['publishedAt'].split('T')[0]
            published_time = video['snippet']['publishedAt'].split('T')[1].split('Z')[0]
            video_link = 'https://www.youtube.com/watch?v='+video['id']['videoId']
            ironhack_videos = ironhack_videos.append({
                            'video_id':video_id,
                            'video_title':video_title, 
                            'published_date':published_date, 
                            'published_time':published_time,
                            'video_link':video_link
                            }, 
                            ignore_index=True)
    return ironhack_videos

## Resulting DataFrame

In [98]:
# executing function to get resulting DataFrame: 
get_videos(video, ironhack_videos)

,video_id,video_title,published_date,published_time,video_link
0,0lRyVSDrY4s,De manucure à développeuse web,2022-06-08,14:59:34,https://www.youtube.com/watch?v=0lRyVSDrY4s
1,jBjBEkUZIdY,Machine learning concepts and no code examples...,2022-06-06,10:49:55,https://www.youtube.com/watch?v=jBjBEkUZIdY
2,sRT9CJvCY54,Intro to Data Storytelling and Data Visualizat...,2022-06-06,10:43:40,https://www.youtube.com/watch?v=sRT9CJvCY54
3,6HO_NI6k7vI,Intro to web scraping : extracting data from T...,2022-06-06,10:27:38,https://www.youtube.com/watch?v=6HO_NI6k7vI
4,gBqfnOV-CIg,Introduction to Decision Trees - Machine Learn...,2022-06-06,10:25:08,https://www.youtube.com/watch?v=gBqfnOV-CIg
5,bXPrHLK2yXI,Python for Office: Automating Company Tasks,2022-06-06,10:15:26,https://www.youtube.com/watch?v=bXPrHLK2yXI
6,au6AHTk7qzQ,Data Analytics 101: Python Programming Basics ...,2022-06-06,10:07:37,https://www.youtube.com/watch?v=au6AHTk7qzQ
7,TdY_Re6eptQ,UX design at TomTom: user centered design in a...,2022-06-06,09:53:30,https://www.youtube.com/watch?v=TdY_Re6eptQ
8,GJVbYLUF2QQ,Conversational UX Design: How to Design for Er...,2022-06-06,09:43:35,https://www.youtube.com/watch?v=GJVbYLUF2QQ
9,d54kz0JO-Kc,Human-centered Design: Beyond Humans - Ironhac...,2022-06-06,09:44:30,https://www.youtube.com/watch?v=d54kz0JO-Kc


## Exporting the data to CSV file

In [99]:
ironhack_videos.to_csv(f'output\ironhack_videos.csv', index = False)